In [1]:
%pip install open_spiel
%pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 9.1 MB/s eta 0:00:00


In [ ]:
import tensorflow.compat.v1 as tf

from open_spiel.python import policy
from open_spiel.python.algorithms import deep_cfr
from open_spiel.python.algorithms import exploitability
import pyspiel
# import deep_cfr_tf2

# Get this working for kuhn_poker
# [*] Add way to track the progress of training
# [ ] Add card encoding to network
# [ ] Tune network size

game = pyspiel.load_game("kuhn_poker")
with tf.Session() as sess:
  deep_cfr_solver = deep_cfr.DeepCFRSolver(
      sess,
      game,
      policy_network_layers=(8, 8, 4),
      advantage_network_layers=(4, 4, 2),
      num_iterations=100,
      num_traversals=1000,
      learning_rate=1e-3,
      batch_size_advantage=None,
      batch_size_strategy=None,
      memory_capacity=1e7)
  sess.run(tf.global_variables_initializer())
  deep_cfr_solver.solve()

  conv = exploitability.nash_conv(
      game,
      policy.tabular_policy_from_callable(game, deep_cfr_solver.action_probabilities),
  )
  print("Deep CFR NashConv: {}".format(conv))
